In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import matplotlib.pylab as pll
from collections import Counter
from datetime import datetime
import time
import random
import math
import os
import itertools
from matplotlib import ticker
import matplotlib.colors as colors
from pylab import rcParams

In [ ]:
slurm1 = pd.read_csv('SlurmPlot2.csv')
slurm1.tail(100)

In [ ]:
slurm1.head(2)

In [ ]:
slurm = slurm1[['UID','JobID','Account','Submittimestamp','Start','End','Eligibletimestamp','Starttimestamp','Endtimestamp','AllocNodes','AllocCPUS','TimelimitRaw', 'NNodes','Submit','State','ExitCode','QOS','QOSRAW','Partition','TRESAllocMem','TRESINMaxUsagemem','TotalCPUtime']]

In [ ]:
slurm = slurm[slurm["Starttimestamp"] != 'Unknown']
slurm = slurm[slurm["Endtimestamp"] != 'Unknown']
slurm['JobLen'] = slurm['Endtimestamp'].astype(float) - slurm['Starttimestamp'].astype(float)
slurm['Totalres'] = slurm['JobLen'] * slurm['NNodes']
slurm['JobID'] = slurm['JobID'].astype(str)

In [ ]:
discard = ["."]
waittime = slurm[~slurm.JobID.str.contains('|'.join(discard),regex = False)]
waittime['Jcount'] = 1
waittime['Waittime'] = waittime['Starttimestamp'].astype(float) - waittime['Submittimestamp'].astype(float)

In [ ]:
waittime['year'] = pd.DatetimeIndex(waittime['Submit']).year
waittime['month'] = pd.DatetimeIndex(waittime['Submit']).month
waittime['year'] = waittime['year'].apply(str)
waittime['month'] = waittime['month'].apply(str)
waittime['yearmonth'] = waittime['year'] + '-' + waittime['month']

In [ ]:
waittime.head()

In [ ]:
waittime = waittime.loc[waittime.JobLen >= 0]

In [ ]:
waittime.info()

In [ ]:
waittime.head(2)

## Histogram of timelimit for each project

In [ ]:
maxxt = waittime.TimelimitRaw.values.max()
meant = waittime.TimelimitRaw.values.mean()
print(maxxt)
print(meant)

In [ ]:
smaller = []
smaller = np.linspace(0, meant, 30,endpoint=False)
smaller = [l.tolist() for l in smaller]
smaller1 = np.linspace(0, smaller[1], 15 ,endpoint=False)
smaller1 = [l.tolist() for l in smaller1]
smaller1 = [ '%.1f' % elem for elem in smaller1 ]
smaller1

In [ ]:
del smaller1[0]

In [ ]:
waittimets = waittime[['Account','TimelimitRaw']]
smaller1 = np.asarray(smaller1, dtype = float)
coun = 0
for idx,row in waittimets.iterrows():
    for index, elem in enumerate(smaller1):
        if waittimets.loc[idx,'TimelimitRaw'] <= smaller1[0]:
            waittimets.loc[idx,'Ts 0-{}'.format(smaller1[0])] = 1    
        else:
            waittimets.loc[idx,'Ts 0-{}'.format(smaller1[0])] = 0   
        prevele = smaller1[index - 1]      
        if waittimets.loc[idx,'TimelimitRaw'] > prevele and waittimets.loc[idx,'TimelimitRaw'] <= elem:
            waittimets.loc[idx,'Ts {}-{}'.format(prevele,elem)] = 1
        else:
            waittimets.loc[idx,'Ts {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
waittimets.head()

In [ ]:
waittimets = waittimets.drop(['Ts 15.8-1.1'], axis=1)
colets = waittimets.columns
colets = colets.tolist()
colets

In [ ]:
cts = waittimets.groupby('Account', as_index=False)[
 'Ts 0-1.1',
 'Ts 1.1-2.3',
 'Ts 2.3-3.4',
 'Ts 3.4-4.5',
 'Ts 4.5-5.6',
 'Ts 5.6-6.8',
 'Ts 6.8-7.9',
 'Ts 7.9-9.0',
 'Ts 9.0-10.1',
 'Ts 10.1-11.3',
 'Ts 11.3-12.4',
 'Ts 12.4-13.5',
 'Ts 13.5-14.6',
 'Ts 14.6-15.8'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in cts.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to Time limit for the Account {}".format(str(count)),size=18)
        width = .5 
        temp.plot(kind = 'bar')
        ax.set_xlabel('Time limit criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_xticklabels([
 '0-44',
 '44-89',
 '89-134',
 '134-179',
 '179-224',
 '224-269',
 '269-314',
 '314-358',
 '358-403',
 '403-448',
 '448-493',
 '493-538',
 '538-583',
 '583-627'],size = 18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("18")
        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_timelimit_sub'):
            os.makedirs('Histogram_timelimit_sub')
        fig.savefig('Histogram_timelimit_sub/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

In [ ]:
partst = []
parts1t = []
parts2t = []
parts3t = []
partst = np.linspace(0, meant, 30,endpoint=False)
parts1t = np.linspace(meant, maxxt, 3)
partst = [l.tolist() for l in partst]
parts1t = [l.tolist() for l in parts1t]
parts2t.append(np.linspace(parts1t[0], parts1t[1], 10,endpoint=False))
parts3t.append(np.linspace(parts1t[1], parts1t[2], 10,endpoint=False))
parts2t = [l2.tolist() for l2 in parts2t]
parts2t = list(np.concatenate(parts2t).flat)
parts3t = [l3.tolist() for l3 in parts3t]
parts3t = list(np.concatenate(parts3t).flat)

In [ ]:
partst = partst + parts2t + parts3t
del partst[0]
partst

In [ ]:
partst = [ '%.2f' % elem for elem in partst ]


In [ ]:
partst

In [ ]:
partst = np.asarray(partst, dtype = float)

In [ ]:
waittimet = waittime[['Account','TimelimitRaw']]
coun = 0
for idx,row in waittimet.iterrows():
    for index, elem in enumerate(partst):
        if waittimet.loc[idx,'TimelimitRaw'] <= partst[0]:
            waittimet.loc[idx,'T 0-{}'.format(partst[0])] = 1    
        else:
            waittimet.loc[idx,'T 0-{}'.format(partst[0])] = 0   
        prevele = partst[index - 1]      
        if waittimet.loc[idx,'TimelimitRaw'] > prevele and waittimet.loc[idx,'TimelimitRaw'] <= elem:
            waittimet.loc[idx,'T {}-{}'.format(prevele,elem)] = 1
        else:
            waittimet.loc[idx,'T {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
waittimet.head()

In [ ]:
waittimet1 = waittimet.drop(['T 28753.35-16.9'], axis=1)
colt = waittimet1.columns
colt = colt.tolist()
colt

In [ ]:
ccc = waittimet1.groupby('Account', as_index=False)[
 'T 0-16.9',
 'T 16.9-33.8',
 'T 33.8-50.7',
 'T 50.7-67.61',
 'T 67.61-84.51',
 'T 84.51-101.41',
 'T 101.41-118.31',
 'T 118.31-135.21',
 'T 135.21-152.11',
 'T 152.11-169.02',
 'T 169.02-185.92',
 'T 185.92-202.82',
 'T 202.82-219.72',
 'T 219.72-236.62',
 'T 236.62-253.52',
 'T 253.52-270.42',
 'T 270.42-287.33',
 'T 287.33-304.23',
 'T 304.23-321.13',
 'T 321.13-338.03',
 'T 338.03-354.93',
 'T 354.93-371.83',
 'T 371.83-388.74',
 'T 388.74-405.64',
 'T 405.64-422.54',
 'T 422.54-439.44',
 'T 439.44-456.34',
 'T 456.34-473.24',
 'T 473.24-490.14',
 'T 490.14-507.05',
 'T 507.05-1993.69',
 'T 1993.69-3480.34',
 'T 3480.34-4966.99',
 'T 4966.99-6453.64',
 'T 6453.64-7940.28',
 'T 7940.28-9426.93',
 'T 9426.93-10913.58',
 'T 10913.58-12400.23',
 'T 12400.23-13886.88',
 'T 13886.88-15373.52',
 'T 15373.52-16860.17',
 'T 16860.17-18346.82',
 'T 18346.82-19833.47',
 'T 19833.47-21320.11',
 'T 21320.11-22806.76',
 'T 22806.76-24293.41',
 'T 24293.41-25780.06',
 'T 25780.06-27266.7',
 'T 27266.7-28753.35'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in ccc.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to Time limit for the Account {}".format(str(count)),size=18)
        width = .5 
        temp.plot(kind = 'bar')
        ax.set_xlabel('Time limit criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_xticklabels( [
 '0-16',
 '16-33',
 '33-50',
 '50-67',
 '67-84',
 '84-101',
 '101-118',
 '118-135',
 '135-152',
 '152-169',
 '169-185',
 '185-202',
 '202-219',
 '219-236',
 '236-253',
 '253-270',
 '270-287',
 '287-304',
 '304-321',
 '321-338',
 '338-354',
 '354-371',
 '371-388',
 '388-405',
 '405-422',
 '422-439',
 '439-456',
 '456-473',
 '473-490',
 '490-507',
 '507-1993',
 '1993-3480',
 '3480-4966',
 '4966-6453',
 '6453-7940',
 '7940-9426',
 '9426-10913',
 '10913-12400',
 '12400-13886',
 '13886-15373',
 '15373-16860',
 '16860-18346',
 '18346-19833',
 '19833-21320',
 '21320-22806',
 '22806-24293',
 '24293-25780',
 '25780-27266',
 '27266-28753'],size = 16)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("16")
        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_timelimit'):
            os.makedirs('Histogram_timelimit')
        fig.savefig('Histogram_timelimit/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

## Timelimit estimation histogram for each project

In [ ]:
waittimef = waittime.loc[(waittime.State == "COMPLETED")]

In [ ]:
waittimef.head(100)

In [ ]:
waittimef['Estimation'] =  waittimef['TimelimitRaw'] / (waittimef['JobLen']/60) 

In [ ]:
for idx, row in waittimef.iterrows():
    if  waittimef.loc[idx,'JobLen'] == 0:
        waittimef.loc[idx,'Estimation'] = waittimef.loc[idx,'TimelimitRaw']

In [ ]:
waittimef = waittimef[['UID','Account','JobID','TimelimitRaw','NNodes','QOS','Partition','JobLen','Jcount','Estimation','month','yearmonth','year']]

In [ ]:
waittimefe = waittimef[['Account','Estimation']]

In [ ]:
waittimefer = waittimef[['Account','Estimation']]

In [ ]:
maxxe = waittimefe.Estimation.values.max()
meane = waittimefe.Estimation.values.mean()
print(maxxe)
print(meane)

In [ ]:
smallerbins = []
smallerbins = np.linspace(0, meane, 30,endpoint=False)
smallerbins = [l.tolist() for l in smallerbins]
smallerbin1 = np.linspace(0, smallerbins[1], 15 ,endpoint=False)
smallerbin1 = [l.tolist() for l in smallerbin1]
smallerbin1 = [ '%.1f' % elem for elem in smallerbin1 ]
smallerbin1

In [ ]:
del smallerbin1[0]

In [ ]:
smallerbin1

In [ ]:
smallerbin1 = np.asarray(smallerbin1, dtype = float)
coun = 0
for idx,row in waittimefer.iterrows():
    for index, elem in enumerate(smallerbin1):
        if waittimefer.loc[idx,'Estimation'] <= smallerbin1[0]:
            waittimefer.loc[idx,'E 0-{}'.format(smallerbin1[0])] = 1    
        else:
            waittimefer.loc[idx,'E 0-{}'.format(smallerbin1[0])] = 0   
        prevele = smallerbin1[index - 1]      
        if waittimefer.loc[idx,'Estimation'] > prevele and waittimefer.loc[idx,'Estimation'] <= elem:
            waittimefer.loc[idx,'E {}-{}'.format(prevele,elem)] = 1
        else:
            waittimefer.loc[idx,'E {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
waittimefer.head()

In [ ]:
waittimefer = waittimefer.drop(['E 17.1-1.2'], axis=1)
coler = waittimefer.columns
coler = coler.tolist()
coler

In [ ]:
cr = waittimefer.groupby('Account', as_index=False)[
 'E 0-1.2',
 'E 1.2-2.4',
 'E 2.4-3.7',
 'E 3.7-4.9',
 'E 4.9-6.1',
 'E 6.1-7.3',
 'E 7.3-8.5',
 'E 8.5-9.8',
 'E 9.8-11.0',
 'E 11.0-12.2',
 'E 12.2-13.4',
 'E 13.4-14.6',
 'E 14.6-15.9',
 'E 15.9-17.1'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in cr.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to time limit estimation for the Account {}".format(str(count)),size=18)
        width = .5 
        temp.plot(kind = 'bar')
        ax.set_xlabel('Estimation criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_xticklabels([
 '0-1',
 '1-2',
 '2-3',
 '3-4',
 '4-6',
 '6-7',
 '7-8',
 '8-9',
 '9-11',
 '11-12',
 '12-13',
 '13-14',
 '14-15',
 '15-17'],size = 18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("18")
        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_Estimation_sub'):
            os.makedirs('Histogram_Estimation_sub')
        fig.savefig('Histogram_Estimation_sub/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

In [ ]:
partse = []
parts1e = []
parts2e = []
parts3e = []
partse = np.linspace(0, meane, 30,endpoint=False)
parts1e = np.linspace(meane, maxxe, 3)
partse = [l.tolist() for l in partse]
parts1e = [l.tolist() for l in parts1e]
parts2e.append(np.linspace(parts1e[0], parts1e[1], 10,endpoint=False))
parts3e.append(np.linspace(parts1e[1], parts1e[2], 10,endpoint=False))
parts2e = [l2.tolist() for l2 in parts2e]
parts2e = list(np.concatenate(parts2e).flat)
parts3e = [l3.tolist() for l3 in parts3e]
parts3e = list(np.concatenate(parts3e).flat)

In [ ]:
partse = partse + parts2e + parts3e
del partse[0]


In [ ]:
partse = [ '%.2f' % elem for elem in partse ]
partse

In [ ]:
partse = np.asarray(partse, dtype = float)

In [ ]:
coun = 0
for idx,row in waittimefe.iterrows():
    for index, elem in enumerate(partse):
        if waittimefe.loc[idx,'Estimation'] <= partse[0]:
            waittimefe.loc[idx,'E 0-{}'.format(partse[0])] = 1    
        else:
            waittimefe.loc[idx,'E 0-{}'.format(partse[0])] = 0   
        prevele = partse[index - 1]      
        if waittimefe.loc[idx,'Estimation'] > prevele and waittimefe.loc[idx,'Estimation'] <= elem:
            waittimefe.loc[idx,'E {}-{}'.format(prevele,elem)] = 1
        else:
            waittimefe.loc[idx,'E {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
waittimefe.head()

In [ ]:
waittimefe = waittimefe.drop(['E 1716867.47-18.31'], axis=1)
cole = waittimefe.columns
cole = cole.tolist()
cole

In [ ]:
cccc = waittimefe.groupby('Account', as_index=False)[
 'E 0-18.31',
 'E 18.31-36.62',
 'E 36.62-54.94',
 'E 54.94-73.25',
 'E 73.25-91.56',
 'E 91.56-109.87',
 'E 109.87-128.18',
 'E 128.18-146.5',
 'E 146.5-164.81',
 'E 164.81-183.12',
 'E 183.12-201.43',
 'E 201.43-219.74',
 'E 219.74-238.06',
 'E 238.06-256.37',
 'E 256.37-274.68',
 'E 274.68-292.99',
 'E 292.99-311.3',
 'E 311.3-329.62',
 'E 329.62-347.93',
 'E 347.93-366.24',
 'E 366.24-384.55',
 'E 384.55-402.86',
 'E 402.86-421.18',
 'E 421.18-439.49',
 'E 439.49-457.8',
 'E 457.8-476.11',
 'E 476.11-494.42',
 'E 494.42-512.74',
 'E 512.74-531.05',
 'E 531.05-549.36',
 'E 549.36-90881.89',
 'E 90881.89-181214.42',
 'E 181214.42-271546.96',
 'E 271546.96-361879.49',
 'E 361879.49-452212.02',
 'E 452212.02-542544.55',
 'E 542544.55-632877.08',
 'E 632877.08-723209.62',
 'E 723209.62-813542.15',
 'E 813542.15-903874.68',
 'E 903874.68-994207.21',
 'E 994207.21-1084539.74',
 'E 1084539.74-1174872.28',
 'E 1174872.28-1265204.81',
 'E 1265204.81-1355537.34',
 'E 1355537.34-1445869.87',
 'E 1445869.87-1536202.4',
 'E 1536202.4-1626534.94',
 'E 1626534.94-1716867.47'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in cccc.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to time limit estimation for the Account {}".format(str(count)),size=18)
        width = .5 
        temp.plot(kind = 'bar')
        ax.set_xlabel('Estimation criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_xticklabels( 
 [
 '0-18',
 '18-36',
 '36-54',
 '54-73',
 '73-91',
 '91-109',
 '109-128',
 '128-146',
 '146-164',
 '164-183',
 '183-201',
 '201-219',
 '219-238',
 '238-256',
 '256-274',
 '274-292',
 '292-311',
 '311-329',
 '329-347',
 '347-366',
 '366-384',
 '384-402',
 '402-421',
 '421-439',
 '439-457',
 '457-476',
 '476-494',
 '494-512',
 '512-531',
 '531-549',
 '549-90881',
 '90881-181214',
 '181214-271546',
 '271546-361879',
 '361879-452212',
 '452212-542544',
 '542544-632877',
 '632877-723209',
 '723209-813542',
 '813542-903874',
 '903874-994207',
 '994207-1084539',
 '1084539-1174872',
 '1174872-1265204',
 '1265204-1355537',
 '1355537-1445869',
 '1445869-1536202',
 '1536202-1626534',
 '1626534-1716867'],size = 16)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("16")

        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_Estimation'):
            os.makedirs('Histogram_Estimation')
        fig.savefig('Histogram_Estimation/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

In [ ]:
waittimef.head(100)

## Estimation of timelimit to actual runtime according to job size 

In [ ]:
waittimee_NN = waittimef.groupby('NNodes', as_index=False).agg({'Estimation':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 60, 10
waittimee_NN['NNodes'] = waittimee_NN['NNodes'].apply(str)
plots = sns.barplot(x ='NNodes',y = 'Estimation',data =  waittimee_NN)
plots.set_title('Mean Estimation according to different job size',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
if not os.path.exists('Timelimit_Analysis'):
    os.makedirs('Timelimit_Analysis')
plt.savefig('Timelimit_Analysis/Timelimit_Analysis_NN.png')

## Estimation of timelimit to actual runtime according to project

In [ ]:
waittimee_A = waittimef.groupby('Account', as_index=False).agg({'Estimation':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 60, 10
waittimee_A['Account'] = waittimee_A['Account'].apply(str)
plots = sns.barplot(x ='Account',y = 'Estimation',data =  waittimee_A)
plots.set_title('Mean Estimation according to different projects',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
if not os.path.exists('Timelimit_Analysis'):
    os.makedirs('Timelimit_Analysis')
plt.savefig('Timelimit_Analysis/Timelimit_Analysis_Account.png')

## Estimation of timelimit to actual runtime according to QOS

In [ ]:
waittimee_QOS = waittimef.groupby('QOS', as_index=False).agg({'Estimation':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
waittimee_QOS['QOS'] = waittimee_QOS['QOS'].apply(str)
plots = sns.barplot(x ='QOS',y = 'Estimation',data =  waittimee_QOS)
plots.set_title('Mean Estimation according to different QOS',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
if not os.path.exists('Timelimit_Analysis'):
    os.makedirs('Timelimit_Analysis')
plt.savefig('Timelimit_Analysis/Timelimit_Analysis_QOS.png')

In [ ]:
waittimef.Estimation.describe()

In [ ]:
waittimeefsdf = waittimef.loc[(waittimef.JobLen/60) > waittimef.TimelimitRaw]

In [ ]:
waittimeefsdf.Estimation.describe()

## Number of jobs according to timelimit estimation by the users for the year

In [ ]:
def funn(row):
    if row['Estimation'] < 1:
        val = 1
    else:
        val = 0
    return val
waittimef['Underest'] = waittimef.apply(funn,axis =1)
def funn1(row):
    if row['Estimation'] <= 2 and row['Estimation'] >= 1:
        val = 1
    else:
        val = 0
    return val
waittimef['uptotwice'] = waittimef.apply(funn1,axis =1)
def funn2(row):
    if row['Estimation'] > 2 and row['Estimation'] <= 10:
        val = 1
    else:
        val = 0
    return val
waittimef['upto10times'] = waittimef.apply(funn2,axis =1)

def funn3(row):
    if row['Estimation'] > 10 and row['Estimation'] <= 100:
        val = 1
    else:
        val = 0
    return val
waittimef['upto100times'] = waittimef.apply(funn3,axis =1)
def funn4(row):
    if row['Estimation'] > 100 and row['Estimation'] <= 1000:
        val = 1
    else:
        val = 0
    return val
waittimef['upto1000times'] = waittimef.apply(funn4,axis =1)
def funn5(row):
    if row['Estimation'] > 1000:
        val = 1
    else:
        val = 0
    return val
waittimef['morethan1000times'] = waittimef.apply(funn5,axis =1)

In [ ]:
def functiont(years):
    coun = 0
    df_list = []
    df_list2 = []
    for i in years:
        temp = waittimef[waittimef.yearmonth.str.contains(i,regex = False)]
        df_list.append(temp)
    for j in df_list:
        temp1 = j.groupby(['month','year'], as_index=False).agg({'Underest':'sum','uptotwice':'sum','upto10times':'sum','upto100times':'sum','upto1000times':'sum','morethan1000times':'sum'})
        temp1['month'] = temp1['month'].apply(int)
        temp1 = temp1.sort_values('month')
        df_list2.append(temp1)
    for l in df_list2:
        count = 0
        num = l.month
        datetime_object = []
        month_name = []
        for m in num:
            datetime_object.append(datetime.strptime(str(m), "%m"))
        for d in datetime_object:
            month_name.append(d.strftime("%b"))
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to timelimit estimation by the users for the year {}".format(str(l.year[0])),size=22)
        width = .5 
        ax.bar(l.month, l.Underest, width, color='g',alpha = 0.5)
        ax.bar(l.month, l.uptotwice, width,bottom =l.Underest, color='b',alpha = 0.5)
        ax.bar(l.month, l.upto10times, width,bottom =l.Underest + l.uptotwice, color='brown',alpha = 0.5)
        ax.bar(l.month, l.upto100times, width,bottom =l.Underest + l.uptotwice + l.upto10times , color='k',alpha = 0.5)
        ax.bar(l.month, l.upto1000times, width,bottom =l.Underest + l.uptotwice + l.upto10times + l.upto100times, color='yellow',alpha = 0.5)
        ax.bar(l.month, l.morethan1000times, width,bottom =l.Underest + l.uptotwice + l.upto10times + l.upto100times + l.upto1000times, color='orange',alpha = 0.8)
        fig.set_figheight(12)
        fig.set_figwidth(16)
        # for p in ax.patches:
        #     ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),fontsize=10)
        ax.set_xlabel('Months', fontsize=18)
        ax.set_ylabel('number of jobs', fontsize=18)
        ax.set(xticks = l.month)
        ax.set_xticklabels(month_name, fontsize=18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("18")
        plt.subplots_adjust(left=0, bottom=0)
        # plt.legend(["Under Estimation",'Estimation upto twice', "Estimation from twice to 10 times","Estimation from 10 to 100 times",'Estimation from 100 to 1000 times','Estimation more than 1000 times'])
        coun = coun + 1
        if not os.path.exists('RSS_Analysis'):
            os.makedirs('Timelimit_Analysis')
        fig.savefig('Timelimit_Analysis/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)
        plt.show()

In [ ]:
years = ['2019', '2020', '2021', '2022']
functiont(years)

In [ ]:
waittimef.head()

In [ ]:
estimationtime = waittimef[['Account','TimelimitRaw','Estimation','JobLen']]

In [ ]:
estimationtime['TimelimitRaw'] = estimationtime['TimelimitRaw'] * 60

In [ ]:
estimationtime.head()

In [ ]:
waittime.head()

## Timelimit vs actual runtime according to resources used for every account.

In [ ]:
waittime['ratio'] =  waittime['JobLen'] /(waittime['TimelimitRaw'] * 60 )

In [ ]:
resources = waittime[['Account','TimelimitRaw','ratio','JobLen','Totalres','Jcount']]

In [ ]:
resources['Timelimit'] = resources['TimelimitRaw']*60

In [ ]:
resources = resources.loc[resources['JobLen'] > 1]

In [ ]:
resources = resources.loc[resources['TimelimitRaw'] >= 0]

In [ ]:
def functionhist(accounts):
    coun = 0
    for i in accounts:
        print(i)
        temp = resources.loc[resources.Account == i] 
        jobs = temp.shape[0]
        rcParams['figure.figsize'] = 15, 15
        import matplotlib.pyplot as plt
        mm=21*24*3600
        fig = plt.figure()
        ax = fig.add_subplot(111)
        heatmap, xedges, yedges = np.histogram2d(x=temp['JobLen'],y=temp['Timelimit'],weights=temp['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
        extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

        print(heatmap.min(),heatmap.max())
        cutoff=10
        pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
        plt.xlabel('Timelimit in seconds')
        plt.ylabel('Runtime in seconds')
        plt.xlim(0,mm)
        plt.ylim(0,mm)
        plt.axis('square')

        plt.title('The user time limit vs. run time according to projects')
        fig.colorbar(pcm, ax=ax, extend='max')
        if not os.path.exists('colorbars_timelimitvsjoblen'):
            os.makedirs('colorbars_timelimitvsjoblen')
        plt.savefig('colorbars_timelimitvsjoblen/{}_clustering_Accounts.png' .format(i),bbox_inches='tight', dpi=100)
        plt.show()

In [ ]:
accounts = resources['Account'].unique()
accounts = accounts.tolist()
functionhist(accounts)

## Timelimit vs actual runtime according to resources used for every partition.

In [ ]:
partitio = waittime[['Account','Partition','TimelimitRaw','ratio','JobLen','Totalres','Jcount']]

In [ ]:
partitio['Timelimit'] = partitio['TimelimitRaw']*60

In [ ]:
partitio = partitio.loc[partitio['JobLen'] > 1]

In [ ]:
def functionpart(partitions):
    coun = 0
    for i in partitions:
        print(i)
        temp = partitio.loc[partitio.Partition == i]  
        jobs = temp.shape[0]
        rcParams['figure.figsize'] = 15, 15
        cm = plt.cm.get_cmap('turbo')
        sc = plt.scatter(temp['Timelimit'], temp['JobLen'],c=temp['Totalres'] ,cmap=cm,s=10)
        # plt.axis('scaled')
        plt.axis('square')
        plt.colorbar(sc,ticklocation = 'right',shrink=0.80,alpha = 0.5)
        plt.title('Clustering for the partition {},jobs {}'.format(i,jobs))
        plt.xlabel('Timelimit')
        plt.ylabel('JobLen')
        if not os.path.exists('part_colorbars_timelimitvsjoblen'):
            os.makedirs('part_colorbars_timelimitvsjoblen')
        plt.savefig('part_colorbars_timelimitvsjoblen/{}_clustering_Partition.png' .format(i),bbox_inches='tight', dpi=100)
        plt.show()

In [ ]:
partitions = partitio['Partition'].unique()
partitions = partitions.tolist()
functionpart(partitions)

## Timelimit vs actual runtime according to resources used for every partition and account.

In [ ]:
def functionpart2(partitions,accounts):
    coun = 0
    for i in partitions:
        print(i)
        temp = partitio.loc[partitio.Partition == i] 
        for a in accounts:
            temp2 = temp.loc[temp.Account == a] 
            if not temp2.empty:
                jobs = temp2.shape[0]
                rcParams['figure.figsize'] = 15, 15
                cm = plt.cm.get_cmap('turbo')
                sc = plt.scatter(temp2['Timelimit'], temp2['JobLen'],c=temp2['Totalres'] ,cmap=cm,s=10)
                # plt.axis('scaled')
                plt.axis('square')
                plt.colorbar(sc,ticklocation = 'right',shrink=0.80,alpha = 0.5)
                plt.title('Clustering for the partition {},account {},jobs {}'.format(i,a,jobs))
                plt.xlabel('Timelimit')
                plt.ylabel('JobLen')
                if not os.path.exists('part2_colorbars_timelimitvsjoblen'):
                    os.makedirs('part2_colorbars_timelimitvsjoblen')
                plt.savefig('part2_colorbars_timelimitvsjoblen/{}_clustering_Partition_{}.png' .format(i,a),bbox_inches='tight', dpi=100)
                plt.show()

In [ ]:
partitions = partitio['Partition'].unique()
accounts = partitio['Account'].unique()
accounts = accounts.tolist() 
partitions = partitions.tolist()
functionpart2(partitions,accounts)

## Timelimit vs actual runtime according to resources used for every job state.

In [ ]:
states = waittime[['Account','State','TimelimitRaw','ratio','JobLen','Totalres','Jcount','NNodes','Partition']]

In [ ]:
waittime.head(1)

In [ ]:
states.head()

In [ ]:
states.loc[states['State'].str.contains('by'), 'State'] = 'CANCELLED BY'

In [ ]:
states['Timelimit'] = states['TimelimitRaw']*60
states = states.loc[states['JobLen'] > 1]

In [ ]:
def functionstate(state):
    coun = 0
    for i in state:
        print(i)
        temp = states.loc[states.State == i]  
        jobs = temp.shape[0]
        rcParams['figure.figsize'] = 15, 15
        cm = plt.cm.get_cmap('turbo')
        sc = plt.scatter(temp['Timelimit'], temp['JobLen'],c=temp['NNodes'] ,cmap=cm,s=10)
        # plt.axis('scaled')
        plt.axis('square')
        plt.colorbar(sc,ticklocation = 'right',shrink=0.80,alpha = 0.5)
        plt.title('Clustering for the state {},jobs {}'.format(i,jobs))
        plt.xlabel('Timelimit')
        plt.ylabel('JobLen')
        if not os.path.exists('state_colorbars_timelimitvsjoblen'):
            os.makedirs('state_colorbars_timelimitvsjoblen')
        plt.savefig('state_colorbars_timelimitvsjoblen/{}_clustering_state.png' .format(i),bbox_inches='tight', dpi=100)
        plt.show()

In [ ]:
state = states['State'].unique()
state = state.tolist()
functionstate(state)

## Timelimit vs actual runtime according to RSS used for every job state.

In [ ]:
states2 = waittimee[['Account','State','TimelimitRaw','JobLen','Totalres','Jcount','NNodes','MaxRSS']]

In [ ]:
states2.loc[states2['State'].str.contains('by'), 'State'] = 'CANCELLED BY'

In [ ]:
states2['Timelimit'] = states2['TimelimitRaw']*60
states2 = states2.loc[states2['JobLen'] > 1]

In [ ]:
def functionrss(state):
    coun = 0
    for i in state:
        print(i)
        temp = states2.loc[states2.State == i]  
        jobs = temp.shape[0]
        rcParams['figure.figsize'] = 15, 15
        cm = plt.cm.get_cmap('turbo')
        sc = plt.scatter(temp['Timelimit'], temp['JobLen'],c=temp['MaxRSS'] ,cmap=cm,s=10)
        # plt.axis('scaled')
        plt.axis('square')
        plt.colorbar(sc,ticklocation = 'right',shrink=0.80,alpha = 0.5)
        plt.title('Clustering for the state {},jobs {}'.format(i,jobs))
        plt.xlabel('Timelimit')
        plt.ylabel('JobLen')
        if not os.path.exists('state2_colorbars_timelimitvsjoblen'):
            os.makedirs('state2_colorbars_timelimitvsjoblen')
        plt.savefig('state2_colorbars_timelimitvsjoblen/{}_clustering_state.png' .format(i),bbox_inches='tight', dpi=100)
        plt.show()

In [ ]:
state = states['State'].unique()
state = state.tolist()
functionrss(state)

## Timelimit vs actual runtime according to resources used for every job state and partition.

In [ ]:
def functionpartstat(partitions,state):
    coun = 0
    for i in partitions:
        print(i)
        temp = states.loc[states.Partition == i] 
        for a in state:
            temp2 = temp.loc[temp.State == a] 
            if not temp2.empty:
                jobs = temp2.shape[0]
                rcParams['figure.figsize'] = 15, 15
                cm = plt.cm.get_cmap('turbo')
                sc = plt.scatter(temp2['Timelimit'], temp2['JobLen'],c=temp2['Totalres'] ,cmap=cm,s=10)
                plt.colorbar(sc,ticklocation = 'right',shrink=0.80,alpha = 0.5)
                plt.title('Clustering for the partition {},State {},jobs {}'.format(i,a,jobs))
                plt.axis('square')
                plt.xlabel('Timelimit')
                plt.ylabel('JobLen')
                if not os.path.exists('partstate_colorbars_timelimitvsjoblen'):
                    os.makedirs('partstate_colorbars_timelimitvsjoblen')
                plt.savefig('partstate_colorbars_timelimitvsjoblen/{}_clustering_Partition_{}.png' .format(i,a),bbox_inches='tight', dpi=100)
                plt.show()

In [ ]:
partitions = states['Partition'].unique()
state = states['State'].unique()
state = state.tolist() 
partitions = partitions.tolist()
functionpartstat(partitions,state)

## Timelimit vs actual runtime according to resources used for every account,job state and partition.

In [ ]:
def function2partstat(accounts,partitions,state):
    coun = 0
    for p in accounts:
        t = states.loc[states.Account == p]
        for i in partitions:
            print(i)
            temp = t.loc[t.Partition == i] 
            for a in state:
                temp2 = temp.loc[temp.State == a] 
                if not temp2.empty:
                    jobs = temp2.shape[0]
                    rcParams['figure.figsize'] = 15, 15
                    cm = plt.cm.get_cmap('turbo')
                    sc = plt.scatter(temp2['Timelimit'], temp2['JobLen'],c=temp2['Totalres'] ,cmap=cm,s=10)
                    plt.axis('square')
                    plt.colorbar(sc,ticklocation = 'right',shrink=0.80,alpha = 0.5)
                    plt.title('Clustering for the account {}, partition {}, State {},jobs {}'.format(p,i,a,jobs))
                    plt.xlabel('Timelimit')
                    plt.ylabel('JobLen')
                    if not os.path.exists('accpartstate_colorbars_timelimitvsjoblen'):
                        os.makedirs('accpartstate_colorbars_timelimitvsjoblen')
                    plt.savefig('accpartstate_colorbars_timelimitvsjoblen/{}{}_clustering_Partition_{}.png' .format(p,i,a),bbox_inches='tight', dpi=100)
                    plt.show()

In [ ]:
partitions = states['Partition'].unique()
state = states['State'].unique()
state = state.tolist() 
partitions = partitions.tolist()
accounts = states['Account'].unique()
accounts = accounts.tolist() 
function2partstat(accounts,partitions,state)

In [ ]:
waittime.head()

In [ ]:
def functionpartition(accounts):
    coun = 0
    for i in accounts:
        print(i)
        temp = waittime.loc[waittime.Partition == i] 
        jobs = temp.shape[0]
        rcParams['figure.figsize'] = 15, 15
        import matplotlib.pyplot as plt
        mm=21*24*3600
        fig = plt.figure()
        ax = fig.add_subplot(111)
        heatmap, xedges, yedges = np.histogram2d(x=temp['JobLen'],y=temp['TimelimitRaw'] * 60,weights=temp['Totalres'] ,bins=200,range=[[0,mm],[0,mm]])
        extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

        print(heatmap.min(),heatmap.max())
        cutoff=10
        pcm=plt.pcolormesh(xedges, yedges, heatmap, cmap='inferno',norm=colors.LogNorm(vmin=heatmap.min()+cutoff, vmax=heatmap.max()+cutoff))
        plt.xlabel('Timelimit in seconds')
        plt.ylabel('Runtime in seconds')
        plt.xlim(0,mm)
        plt.ylim(0,mm)
        plt.axis('square')

        plt.title('The user time limit vs. run time according to projects')
        fig.colorbar(pcm, ax=ax, extend='max')
        if not os.path.exists('Partition_runtime'):
            os.makedirs('Partition_runtime')
        plt.savefig('Partition_runtime/{}_Partition_runtime.png' .format(i),bbox_inches='tight', dpi=100)
        plt.show()

In [ ]:
partition = waittime['Partition'].unique()
partition = partition.tolist()
functionpartition(partition)